In [1]:
import matplotlib
matplotlib.use('Agg')  # noqa
import matplotlib.pyplot as plt

In [2]:
def forward_euler(y0, delta_t, dydt):
    y_new = y0 + delta_t * dydt
    return y_new

In [3]:
def generate_t_list(t0, dt, tmax):
    t_list = []
    # If 0<dt<1, can't use range function
    if (dt > 0 and dt < 1):
        count = t0
        while (count < tmax):
            t_list.append(count)
            count += dt
    else:  # Get list of times using range
        t_list = list(range(t0, tmax, dt))
    return t_list

In [4]:
def simulate_malaria_simple(SH_0, IH_0, EH_0, RH_0, SM_0, IM_0, mu_H, mu_M, beta_H, beta_M, nu, sigma, gamma, delta, psi, dt, tmax):
    '''
    Numerical simulation of a simple malaria model
    - Uniform birth/death
    - No compartments (Only human and mosquito)
    - Calculated using absolute numbers, not proportions
    
    Humans are simulated using an SEIR model
    Mosquitoes are simulated using an SI model
    Time is in units of days
    
    Variables:
    mu_H = Natural human death rate
    mu_M = Natural mosquito death rate
    beta_H = Human exposure rate (mosquito biting rate * chance of infection per bite)
    beta_M = Mosquito infection rate (mosquito biting rate * chance of infection after bite)
    nu = Tuning factor to scale probability that exposed individuals can pass infection to mosquito
    sigma = Rate of development of clinical symptoms in humans after exposure
    gamma = Human recovery rate
    delta = Infection induced mortality rate in humans
    psi = Human loss of immunity rate (return to susceptible)
    '''
    NH = RH_0 + SH_0 + IH_0 + EH_0
    
    # Storage of values over time
    SH = [SH_0]
    EH = [EH_0]
    IH = [IH_0]
    RH = [RH_0]
    
    SM = [SM_0]
    IM = [IM_0]
    
    # Set initial values
    curr_SH = SH_0
    curr_EH = EH_0
    curr_IH = IH_0
    curr_RH = RH_0
    
    curr_SM = SM_0
    curr_IM = IM_0
    
    # If 0<dt<1, can't use range function
    t_list = generate_t_list(t0=0, dt=dt, tmax=tmax)

    for t in t_list:
        # dSH = birth + unrecovery rate - exposure rate - death rate
        dSH = (mu_H * NH + delta * curr_IH) + (psi * curr_RH) - (beta_H * curr_SH * curr_IM / NH) - (mu_H * curr_SH)
        
        # dEH = Exposure rate - infection rate - death rate
        dEH = (beta_H * curr_SH * curr_IM / NH) - (sigma * curr_EH) - (mu_H * curr_EH)
        
        # dIH = Infection rate - recovery rate - natural death rate - disease associated mortality
        dIH = (sigma * curr_EH) - (gamma * curr_IH) - ((mu_H + delta) * curr_IH)
        
        # dRH = Recovery rate - unrecovery rate - death rate
        dRH = (gamma * curr_IH) - (psi * curr_RH) - (mu_H * curr_RH)
        
        # dSM = Birth - infection rate - death rate
        dSM = (mu_M * (curr_SM + curr_IM)) - (beta_M * (nu * curr_EH + curr_IH)) - (mu_M * curr_SM)
        
        # dIM = Infection rate - death rate
        dIM = (beta_M * (nu * curr_EH + curr_IH)) - (mu_M * curr_IM)

        curr_SH = forward_euler(curr_SH, dt, dSH)
        curr_EH = forward_euler(curr_EH, dt, dEH)
        curr_IH = forward_euler(curr_IH, dt, dIH)
        curr_RH = forward_euler(curr_RH, dt, dRH)
        
        curr_SM = forward_euler(curr_SM, dt, dSM)
        curr_IM = forward_euler(curr_IM, dt, dIM)

        SH.append(curr_SH)
        EH.append(curr_EH)
        IH.append(curr_IH)
        RH.append(curr_RH)
        
        SM.append(curr_SM)
        IM.append(curr_IM)
    
    # Complete list of times, return results
    t_list.append(t_list[-1] + dt)

    return t_list, SH, EH, IH, RH, SM, IM

In [5]:
SH_0 = 4999900
EH_0 = 100
IH_0 = 0
NH = 0
SM_0 = 2000
IM_0 = 0
mu_H = 0.00004
mu_M = 0.05
beta_H = 0.0905 
beta_M = 0.415
nu = 0.5
sigma = 0.09333
gamma = 0.0023 
delta =  0.00174
psi =  0.0027
dt = 1
tmax = 720

t_list, SH, EH, IH, RH, SM, IM = simulate_malaria_simple(SH_0, IH_0, EH_0, NH, SM_0, IM_0, mu_H, mu_M, beta_H, beta_M, nu, sigma, gamma, delta, psi, dt, tmax)

In [6]:
print(len(t_list))
print(len(SH))

721
721


In [7]:
fig, ax = plt.subplots()
ax.plot(t_list,SH,label='S Human')
ax.plot(t_list,EH,label='E Human')
ax.plot(t_list,IH,label='I Human')
ax.plot(t_list,RH,label='R Human')
ax.set_xlabel('Time (Days)')
ax.set_ylabel('Population')
ax.set_title('Human Population vs Time')
ax.legend(loc='lower right')
plt.savefig('smm_humans.png')

In [8]:
fig, ax = plt.subplots()
ax.plot(t_list,SM,label='S Mosquito')
ax.plot(t_list,IM,label='I Mosquito')
ax.set_xlabel('Time (Days)')
ax.set_ylabel('Mosquito Population')
ax.set_title('Mosquito Population vs Time')
ax.legend(loc='lower right')
plt.savefig('smm_mosquito.png')